# Analisis de ventas


In [ ]:
import os
import pandas as pd

## consolidar datos de todo el año

In [ ]:
path = 'Sales_Data'
archivos = [file for file in os.listdir(path) if not file.startswith('.')]
print(archivos)
data_anual = pd.DataFrame()

for archivo in archivos:
    actual = pd.read_csv(path + '/' + archivo)
    data_anual = pd.concat([data_anual, actual])

data_anual.to_csv('consolidado_2019.csv', index=False)

In [ ]:
data_anual.describe()

In [ ]:
data_anual.tail()

## Limpieza de los datos

In [ ]:
# borrar filas vacias
nan_df = data_anual[data_anual.isna().any(axis=1)]

data_anual = data_anual.dropna(how='all')

In [ ]:
nan_df2 = data_anual[data_anual.isna().any(axis=1)]
nan_df2

In [ ]:
data_anual = data_anual[data_anual['Order Date'].str[0:2] != 'Or']

### convertir datos numericos


In [ ]:
data_anual['Quantity Ordered'] = pd.to_numeric(data_anual['Quantity Ordered'])
data_anual['Price Each'] = pd.to_numeric(data_anual['Price Each'])

In [ ]:
type(data_anual['Price Each'][0].values[0])

### agregar columna de mes

In [ ]:
data_anual['Month'] = pd.to_numeric(data_anual['Order Date'].str[:2])
data_anual.head()

In [ ]:
def get_ciudad(direccion):
    return direccion.split(',')[1].strip(' ')
def get_estado(direccion):
    return direccion.split(',')[2].split(' ')[1]

In [ ]:
get_ciudad('755 Sunset St, New York City, NY 10001')

In [ ]:
data_anual['City'] = data_anual['Purchase Address'].apply(get_ciudad)
data_anual.head()

## Exploracion de datos


### cual fue el mejor mes en ventas

In [ ]:
data_anual['Sales'] = data_anual['Quantity Ordered'].astype('int') * data_anual['Price Each'].astype('float')
data_anual.head()

In [ ]:
data_anual.groupby(['Month']).sum()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
months = range(1,13)

plt.bar(months, data_anual.groupby(['Month']).sum()['Sales'])
plt.xticks(months)
plt.ylabel('Ventas en Bs')
plt.xlabel('Mes')


### cual es la ciudad con mas ventas?


In [ ]:
data_anual.groupby(['City']).sum()

In [ ]:
ciudades = [ciudad for ciudad, df in data_anual.groupby(['City'])]
plt.bar(ciudades, data_anual.groupby(['City']).sum()['Sales'])
plt.xticks(ciudades, rotation='vertical')
plt.ylabel('Ventas en $us')
plt.xlabel('Ciudad')

### a que hora deberiamos mostrar publicidad para mejorar nuestras ventas?

In [ ]:
data_anual['Hour'] = pd.to_datetime(data_anual['Order Date']).dt.hour
data_anual.head()

In [ ]:
data_anual['Count'] = 1
data_anual.head(15)

In [ ]:
data_anual.groupby(['Hour']).count()

In [ ]:
horas = [hora for hora, df in data_anual.groupby(['Hour'])]
plt.plot(horas, data_anual.groupby(['Hour']).count()['Count'])
plt.xticks(horas, rotation='vertical')
plt.ylabel('Numero de ordenes')
plt.xlabel('Hora')
plt.grid()

### cuales son los productos que se venden mas a menudo juntos?

In [ ]:
df = data_anual[data_anual['Order ID'].duplicated(keep=False)]
df['Grouped'] = df.groupby('Order ID')['Product'].transform(lambda x: ', '.join(x))
df.head()

In [ ]:
df2 = df[['Order ID', 'Grouped']].drop_duplicates()
df2.tail(50)

In [ ]:
from itertools import combinations
from collections import Counter

contador = Counter()

for fila in df2['Grouped']:
    lista_prod = fila.split(',')
    contador.update(Counter(combinations(lista_prod, 2)))

# menos comunes
contador.most_common()